# Lab 5: Land Surface Temperature using CORINE-based Emissivity

## 🎯 Objectives
In this exercise, you will:
- Select a cloud-free Landsat 8 images from 2013 and 2023 (or different if you're sure that you'll spot a difference in land cover)
- Calculate Brightness Temperature (TB) from Band 10.
- Load CORINE Land Cover data and assign emissivity values to each land cover class.
- Use the Planck-based formula to calculate Land Surface Temperature (LST).
- Visualize and interpret the results.

## Step 1: Define Area of Interest (AOI)
- Use coordinates around Reduta street in Kraków.
- Create a polygon or rectangle using `ee.Geometry.Polygon`.

In [1]:
# TODO: Define AOI
import ee
ee.Authenticate()
ee.Initialize(project='ee-kamilmolo13')

aoi = ee.Geometry.Polygon([
        [19.973, 50.094], 
        [19.973, 50.101],  
        [19.988, 50.101],  
        [19.988, 50.094],  
        [19.973, 50.094]  
    ])

## Step 2: Load Landsat 8 imagery for the dates you've picked
- Filter for low cloud cover (< 20%)
- Select Band 10 and convert to TB using: `TB = ST_B10 * 0.00341802 + 149.0`

In [2]:
# Load image and calculate TB
image_2013 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate('2013-07-26', '2013-08-09') \
    .sort('CLOUDY_PIXEL_PERCENTAGE') \
    .first()

image_2024 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate('2023-06-18', '2023-07-09') \
    .sort('CLOUDY_PIXEL_PERCENTAGE') \
    .first()
TB_im13 = image_2013.select('ST_B10').multiply(0.00341802).add(149).rename('TB')
TB_im24 = image_2024.select('ST_B10').multiply(0.00341802).add(149).rename('TB')

## Step 3: Load CORINE Land Cover data
- Use dataset `COPERNICUS/CORINE/V20/100m/2018`
- Clip it to your AOI

In [3]:
# Load CORINE
corine_image = ee.Image('COPERNICUS/CORINE/V20/100m/2018')

corine_image = corine_image.clip(aoi)

print(corine_image.getInfo())

{'type': 'Image', 'bands': [{'id': 'landcover', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [13, 10], 'origin': [41320, 24526], 'crs': 'EPSG:3035', 'crs_transform': [100, 0, 900000, 0, -100, 5500000]}], 'version': 1578598451411424, 'id': 'COPERNICUS/CORINE/V20/100m/2018', 'properties': {'system:footprint': {'type': 'Polygon', 'coordinates': [[[19.973, 50.094], [19.987999999999996, 50.094], [19.987999999999996, 50.101], [19.973, 50.101], [19.973, 50.094]]]}, 'system:time_start': 1483228800000, 'landcover_class_names': ['Artificial surfaces; urban fabric; continuous urban fabric', 'Artificial surfaces; urban fabric; discontinuous urban fabric', 'Artificial surfaces; industrial, commercial, and transport units; industrial or commercial units', 'Artificial surfaces; industrial, commercial, and transport units; road and rail networks and associated land', 'Artificial surfaces; industrial, commercial, and transport units; port areas', 'A

## Step 4: Assign emissivity to CORINE classes
- Use a dictionary for classes
- Use `remap()` and optionally a default value

In [4]:
import numpy as np
# Create emissivity image
emissivity_dict = {
    111: 0.92,  # Continuous urban fabric
    112: 0.92,  # Discontinuous urban fabric
    121: 0.91,  # Industrial or commercial units
    211: 0.96,  # Non-irrigated arable land
    311: 0.98,  # Forests
    412: 0.97,  # Peat bogs
    324: 0.96,  # Transitional woodland-shrub
    231: 0.97   # Pastures
}

keys = list(emissivity_dict.keys())
values = list(emissivity_dict.values())
emissivity_image = corine_image.remap(keys, values)


## Step 5: Calculate LST using the formula:
$$
LST = \frac{T_B}{1 + \left( \frac{\lambda \cdot T_B}{c_2} \right) \cdot \ln(\varepsilon)}
$$
- λ = 10.8 µm
- c₂ = 14388 µm·K

In [5]:
# Calculate LST
lambda_ = 10.8
c2 = 14388

lst_image = TB_im13.expression(
    'TB / (1 + ((lambda * TB) / c2) * log(1.0 / emissivity))', {
        'TB': TB_im13,
        'emissivity': emissivity_image,
        'lambda': lambda_,
        'c2': c2
    }).rename('LST')

## Step 6: Visualize the LST result
- Use palette: `['blue', 'yellow', 'red']`
- Suggested range: `min=290`, `max=325`

In [6]:
import geemap
# Visualize LST
Map = geemap.Map()
Map.centerObject(aoi, zoom=15)

vis_params = {
    'palette': ['blue', 'yellow', 'red'],
    'min': 290,
    'max': 325,
}

Map.addLayer(lst_image, vis_params, 'LST')
Map

Map(center=[50.097500156313984, 19.980500000000532], controls=(WidgetControl(options=['position', 'transparent…

## Step 7: (Optional) Analyze statistics by land cover class

In [ ]:
# Optionally compute zonal statistics

## Step 8: (Optional - Easter Egg :)) Generate your own Land Cover Classification using TerraTorch and foundation models*

Based on the example/tutorial: https://aiforgood.itu.int/event/workshop-earth-observation-foundation-models-with-prithvi-eo-2-0-and-terratorch/

*to earn 5.0 grade that will make a great impact on your final grade